<a href="https://colab.research.google.com/github/rpezoa/mlvalpo/blob/main/regresionMulti_MLvalpo_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción de temperatura efectiva y velocidad de rotación en estrellas masivas
***
**Objetivo de la actividad**: Usar diversos métodos ML para predecir la temperatura efectiva y velocidad de rotación de espectros de estrellas masivas.

**Datos**: Un subconjunto de espectros de estrellas Be, generados por PhD (c) Daniela Turis (IFA-UV) con el código ZPEKTR.

Al usar ML, este problema se resuelve como un problema de **regresión de múltiples salidas**.

Información obre estrellas masivas en: https://massivestars.ifa.uv.cl/


## Bibliotecas de Python
***

In [ ]:
import os
import sys

import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import numpy as np
import time


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns

## Funciones propias
***

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def plot_results(y_test, y_pred):
    targets = ["T_eff", "v sin i"]

    # Reporte de métricas
    for i, target in enumerate(targets):
        print(f"{target} – Linear Regression")
        print(f"  MSE: {mean_squared_error(y_test.values[:, i], y_pred[:, i]):.2f}")
        print(f"  MAE: {mean_absolute_error(y_test.values[:, i], y_pred[:, i]):.2f}")
        print(f"  R²:  {r2_score(y_test.values[:, i], y_pred[:, i]):.3f}\n")

    # Visualización conjunta
    plt.figure(figsize=(12, 5))  # más ancho que alto

    for i, target in enumerate(targets):
        plt.subplot(1, 2, i+1)
        sns.scatterplot(x=y_test.values[:, i], y=y_pred[:, i], alpha=0.4)
        plt.plot(
            [y_test.values[:, i].min(), y_test.values[:, i].max()],
            [y_test.values[:, i].min(), y_test.values[:, i].max()],
            'r--'
        )
        plt.xlabel(f"True {target}")
        plt.ylabel(f"Predicted {target}")
        plt.title(f"{target} – Linear Regression")
        plt.grid(True)

    plt.tight_layout()
    plt.show()


## Descarga de Datos y Pandas
***
* Datos generados con ZPEKTR, por PhD (c) Daniela Turis IFA-UV.
* Debe obtener el conjunto de espectros llamados: df_ZPEKTR_limb_lineal.csv

In [ ]:
# Dataset provided by PhD (c) Daniela Turis, IFA-UV.

# Verifica si estás en Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # En Colab, usamos gdown
    import gdown
    !gdown https://drive.google.com/uc?id=1m_GajQqDRcKrH8_ExG_0Yp_sQ4MrhZbN
    data=pd.read_csv("df_ZPEKTR_limb_lineal.csv")

else:
    # En entorno local, carga desde disco
    file_path = "/Users/rpezoa/Downloads/df_ZPEKTR_limb_lineal.csv"
    if os.path.exists(file_path):
        print("Archivo cargado localmente:", file_path)
        data=pd.read_csv(file_path)

    else:
        print("No se encontró el archivo local. Debes descargarlo manualmente.")


Downloading...
From: https://drive.google.com/uc?id=1m_GajQqDRcKrH8_ExG_0Yp_sQ4MrhZbN
To: /content/df_ZPEKTR_limb_lineal.csv
100% 19.1M/19.1M [00:00<00:00, 127MB/s]


## Revisar los datos
---
* Revise los datos, vea cuántas columnas tiene, los nombres de las columnas.
* Los parámetros que vamos a predecir serán \<Teff\> y  vsini
* Puede usar las funciones: data.columns, data.info(), data.describe(), data["vsini"].mean(), etc.

In [ ]:
data.head()

### Generación de matrices con formato para ser usados en modelos ML
***
* Lo usual, cuando se trabaja con datos tabulados, es generar una matriz $X$ con los datos de entrada y una matriz $y$ con la salida
* Acá, $X$ contiene los $n=10691$ espectros e $y$ contiene los $n$ valores de salida o etiquetas de cada espectro. En este caso, y tendrá los valores de la temperatura efectiva y velocidad de rotación (vsini).

In [ ]:
X = data.iloc[:,0:170] # X matrix containing the flux of the spectral lines
y_input = data.iloc[:,170:176] # input parameters of ZPEKTR
y_output = data.iloc[:,176:] # output parameters of ZPEKTR
y = data[["<Teff>", "vsini"]] # y matrix containing the values we want to predict

In [ ]:
y

In [ ]:
lambdas = np.array(data.columns[:-18]).astype("float")
plt.figure()
plt.plot(lambdas, X.iloc[2,:])
plt.title("Example of spectrum")
plt.xlabel("Wavelength")
plt.ylabel("Flux")
plt.grid()
plt.show()


### Conjuntos de training, validation y testing
***
* La generación de los conjuntos de training, testing y validation, es fundamental para el entrenamiento y generalización  de los modelos.


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state=0)

## Modelos de machine learning
***
Usaremos cuatro técnicas para predecir la temperatura efectiva y velocidad de rotación:
 * regresión lineal
 * decision tree
 * random forest
 * gradient boosting

### Regresión Lineal

* Acá usaremos el método LinearRegression() de scikit-learn
*

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt


model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)
plot_results(y_test,y_pred_lr)

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor




## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor



### Optmización de hiperparámetros

* Uso de [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) de scikit-learn de Python.

In [ ]:
param_grid = {
    'n_estimators': [10,20],
    #'max_features': ['auto', 'sqrt', 'log2'],
    #'max_depth' : [4,5,6,7,8],
    #'criterion' :['gini', 'entropy']
}

rfc= RandomForestRegressor(random_state=42)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, n_jobs=-1)
CV_rfc.fit(X_train, y_train)

## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor

base_model = GradientBoostingRegressor(n_estimators=10, random_state=42)
model_gb = MultiOutputRegressor(base_model)



In [ ]:
base_estimator = GradientBoostingRegressor(random_state=42)

# Wrap it with MultiOutputRegressor
multi_output_model = MultiOutputRegressor(estimator=base_estimator)


param_grid = {
    'estimator__n_estimators': [5,10,20,40],
    'estimator__max_depth': [None, 10, 20]
}

# Create GridSearchCV object
grid_search = GridSearchCV(
    estimator=multi_output_model,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

# Ahora probemos en líneas observadas
***

## Descarga de líneas observadas

In [ ]:
!gdown
if IN_COLAB:
    # En Colab, usamos gdown
    import gdown
    !gdown https://drive.google.com/uc?id=1WGq1GXOD3SnI985wgMnV7u2NyV3l2BQc
    #!gdown https://drive.google.com/uc?id=1codpDGdi9Z-JjzsJqhu0mxQDCY22XNm5
    #!gdown https://drive.google.com/uc?id=1KQ0mYeRgD_A9u0QyjCMAh1YjakyaE23K
    #!gdown https://drive.google.com/uc?id=1-08NwTp8fFFgCeUj8x9qmg9sfSorZyqt
    df=pd.read_csv("hd60606_4460_4478.dat", header=None,sep=" ")

else:
    # En entorno local, carga desde disco
    file_path = "/Users/rpezoa/repo_temp/mlvalpo/observed_DT/hd60606_4460_4478.dat"
    if os.path.exists(file_path):
        print("Archivo cargado localmente:", file_path)
        df = pd.read_csv(file_path, header=None, sep=" ")

    else:
        print("No se encontró el archivo local. Debes descargarlo manualmente.")

#### Etiquetas de los datos
| Estrella | Teff | log g | v sin i|
|---|---|---|---|
| HD 60606 | 19095 ± 656 | 3.94 ± 0.07 | 298 ± 13 |
| HD 212076 | 17374 ± 631 | 3.86 ± 0.15 | 112 ± 8 |
| HD 158427 | 18668 ± 1395 | 4.31 ± 0.15 | 328 ± 15 |
| HD68980 | 18104 ± 1618 | 3.32 ± 0.33 | 161 ± 13 |


### Preprocesamiento breve
* Acá renombraremos las columnas, solo para mayor claridad


In [ ]:
df.rename(columns={0: 'wavelength', 1:'flux'}, inplace=True)
df.columns

Index(['wavelength', 'flux'], dtype='object')

### Revisar y graficar la línea observada


In [ ]:
df

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(df['wavelength'], df['flux'], color="navy", lw=1.5)
plt.xlabel("Wavelength [Å]")
plt.ylabel("Flux")
plt.title("Espectro Observado - HD 60606 (4460 - 4477 Å)")
plt.grid(True)
plt.tight_layout()
# Guardar a archivo PNG
plt.savefig("spectrum_hd60606_4460_4477.png", dpi=300)
plt.show()

In [ ]:
espectro_obs  = df["flux"]
espectro_obs = espectro_obs.values.reshape(1,-1)

## Predicción con Linear Regresion

## Predicción con Decision Tree

## Predicción con Random Forest

## Predicción con Gradient Boosting

# Preguntas
1. Analice los valores predichos por cada método ML. ¿Qué pasa con la predicción generada por la técnica regresión lineal?

2. ¿Cómo puede mejorar los resultados de Gradient Boosting?
